In [1]:
%load_ext sql

In [2]:
%sql postgresql://appdev@data:5432/appdev

'Connected: appdev@appdev'

In [9]:
%sql SELECT * from Information_schema.tables WHERE table_schema = 'f1db';

13 rows affected.


table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
appdev,f1db,circuits,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructorresults,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructors,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructorstandings,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,drivers,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,driverstandings,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,laptimes,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,pitstops,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,qualifying,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,races,BASE TABLE,None,None,None,None,None,YES,NO,None


In [201]:
%sql SELECT * from drivers WHERE driverref like '%massa%';

1 rows affected.


driverid,driverref,number,code,forename,surname,dob,nationality,url
13,massa,19,MAS,Felipe,Massa,1981-04-25,Brazilian,http://en.wikipedia.org/wiki/Felipe_Massa


In [406]:
%%sql SELECT drivers.forename, drivers.surname, drivers.driverid, count(results.rank) as Wins FROM drivers
JOIN results USING(driverid) WHERE results.rank = 1 GROUP BY driverid ORDER BY Wins DESC;

31 rows affected.


forename,surname,driverid,wins
Kimi,Räikkönen,8,41
Lewis,Hamilton,1,38
Sebastian,Vettel,20,31
Fernando,Alonso,4,22
Michael,Schumacher,30,21
Nico,Rosberg,3,20
Mark,Webber,17,18
Felipe,Massa,13,15
Daniel,Ricciardo,817,9
Jenson,Button,18,8


In [11]:
%sql SELECT * from results limit 1

1 rows affected.


resultid,raceid,driverid,constructorid,number,grid,position,positiontext,positionorder,points,laps,time,milliseconds,fastestlap,rank,fastestlaptime,fastestlapspeed,statusid
1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [408]:
%%sql 
SELECT drivers.driverref, constructors.name, count(*) from drivers
JOIN results USING(driverid)
JOIN constructors USING(constructorid) 
GROUP BY drivers.driverref, constructors.name 
ORDER BY count DESC LIMIT 5;

5 rows affected.


driverref,name,count
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136


In [410]:
%%sql 
SELECT drivers.driverref, constructors.name, count(constructors.name) from drivers
JOIN results USING (driverid) 
JOIN constructors USING (constructorid) 
WHERE results.statusid = 1 
GROUP BY drivers.driverref, constructors.name 
ORDER BY count DESC LIMIT 5;

5 rows affected.


driverref,name,count
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97


In [320]:
%%sql CREATE VIEW pit as
SELECT driverid, raceid, sum(milliseconds) as ms FROM pitstops GROUP BY driverid, raceid;

Done.


[]

In [412]:
%sql SELECT * from pit LIMIT 5

5 rows affected.


driverid,raceid,ms
815,969,22045
8,942,59281
20,898,24086
8,913,89060
807,894,47766


In [413]:
%%sql CREATE VIEW q3 as
SELECT drivers.driverref, constructors.name, count(*), sum(ms)  from drivers
JOIN results USING (driverid) 
FULL OUTER JOIN pit USING (driverid, raceid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1 
GROUP BY drivers.driverref, constructors.name
ORDER BY count DESC

(psycopg2.ProgrammingError) relation "q3" already exists
 [SQL: 'CREATE VIEW q3 as\nSELECT drivers.driverref, constructors.name, count(*), sum(ms)  from drivers\nJOIN results USING (driverid) \nFULL OUTER JOIN pit USING (driverid, raceid)\nJOIN constructors USING (constructorid)\nWHERE results.statusid = 1 \nGROUP BY drivers.driverref, constructors.name\nORDER BY count DESC']


In [415]:
%sql SELECT * from q3 LIMIT 10;

10 rows affected.


driverref,name,count,sum
michael_schumacher,Ferrari,141,None
massa,Ferrari,112,2787290
rosberg,Mercedes,109,11501262
webber,Red Bull,100,3017372
vettel,Red Bull,97,4927545
raikkonen,Ferrari,92,3608658
button,McLaren,87,8515623
alonso,Ferrari,87,3691856
coulthard,McLaren,85,None
hamilton,Mercedes,85,12147994


In [384]:
%sql SELECT * from q3 WHERE driverref like '%l_schumacher' LIMIT 10

3 rows affected.


driverref,name,count,sum
michael_schumacher,Ferrari,141,None
michael_schumacher,Benetton,44,None
michael_schumacher,Mercedes,34,1243146


In [376]:
%sql SELECT count(*) from q3

1 rows affected.


count
658


In [393]:
%%sql CREATE VIEW q3v2 as
SELECT drivers.driverref, constructors.name, count(*), sum(ms)  from drivers
JOIN results USING (driverid) 
JOIN pit USING (driverid, raceid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY drivers.driverref, constructors.name
ORDER BY count DESC

(psycopg2.ProgrammingError) column "totalpitstop" does not exist
LINE 2: ...tructors.name, count(*), sum(ms) as TotalPitStop, TotalPitSt...
                                                             ^
 [SQL: 'CREATE VIEW q3v2 as\nSELECT drivers.driverref, constructors.name, count(*), sum(ms) as TotalPitStop, TotalPitStop/count(results.driverid)  from drivers\nJOIN results USING (driverid) \nJOIN pit USING (driverid, raceid)\nJOIN constructors USING (constructorid)\nWHERE results.statusid = 1\nGROUP BY drivers.driverref, constructors.name\nORDER BY count DESC']


In [417]:
%sql SELECT * from q3v2 LIMIT 5

5 rows affected.


driverref,name,count,sum
rosberg,Mercedes,95,11501262
hamilton,Mercedes,85,12147994
button,McLaren,71,8515623
alonso,Ferrari,70,3691856
vettel,Red Bull,70,4927545


In [419]:
%%sql
SELECT drivers.driverref, constructors.name, count(*), sum(ms) as TotalPitStop, ROUND(sum(ms)/count(results.raceid), 2) as AveragePit  from drivers
JOIN results USING (driverid) 
JOIN pit USING (driverid, raceid)
JOIN constructors USING (constructorid)
WHERE results.statusid = 1
GROUP BY drivers.driverref, constructors.name
ORDER BY averagePit ASC LIMIT 10

10 rows affected.


driverref,name,count,totalpitstop,averagepit
ambrosio,Lotus F1,1,21962,21962.00
rosa,HRT,1,22272,22272.00
hulkenberg,Renault,5,186700,37340.00
glock,Marussia,2,82876,41438.00
bruno_senna,Williams,13,574413,44185.62
perez,Sauber,15,681053,45403.53
resta,Force India,34,1563408,45982.59
ricciardo,Toro Rosso,22,1015872,46176.00
raikkonen,Lotus F1,33,1532336,46434.42
maldonado,Williams,21,982946,46806.95
